In [11]:
import pandas as pd
import numpy as np
import json
import os

In [13]:
from google.colab import files
uploaded = files.upload()

Saving Articles-with-extracted-info.tsv to Articles-with-extracted-info.tsv


In [5]:

uploaded = files.upload()

The data was downloaded from the <a href=http://gun-violence.org/> Gun Violence Database</a> which <a href=http://gun-violence.org/about>claims</a> to automatically search local newspapers for articles about gun violence and uses volunteers to read the artices and update the database. They make available data to be <a href=http://gun-violence.org/download>downloaded</a>. In all there are 2 TSV files. **Events.tsv** and **Articles-with-extracted-info.tsv** 

In [14]:
def read_tsv(file, parse_dates=False, location=False):
    '''
    Function to read in tab delimited files and create a dataframe with the option of using pandas functionality to parse dates from the user defined column
    and option to filter by locatiion
    '''
    events = pd.read_csv(file,sep ='\t',parse_dates = parse_dates)
    if location:
        events = events.loc[events['Address'].str.contains(location)].reset_index()
        
    return events


In [15]:
no_parse = read_tsv('Events.tsv')
events = read_tsv('Events.tsv',['Date'])
phl_events = read_tsv('Events.tsv',['Date'], 'Philadelphia')
print(no_parse.info())
print(events.info())
print(phl_events.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3525 entries, 0 to 3524
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Address                            3525 non-null   object
 1   Date                               3525 non-null   object
 2   Info about time, type of gun etc.  3525 non-null   object
 3   Info about participants            3525 non-null   object
dtypes: object(4)
memory usage: 110.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3525 entries, 0 to 3524
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   Address                            3525 non-null   object             
 1   Date                               3525 non-null   datetime64[ns, UTC]
 2   Info about time, type of gun etc.  3525 non-null   obj

In [16]:
phl_events.head()


,index,Address,Date,"Info about time, type of gun etc.",Info about participants
0,37,"Philadelphia, PA, USA",2015-12-06 00:00:00+00:00,"{""clock-time"":{""endIndex"":177,""startIndex"":170...","[{u'injured': True, u'name': u'', u'hospitaliz..."
1,48,"Kingsessing, Philadelphia, PA 19143, USA",2016-01-31 00:00:00+00:00,"{""clock-time"":{""endIndex"":-1,""startIndex"":-1,""...","[{u'injured': True, u'name': u'', u'hospitaliz..."
2,50,"Philadelphia, PA, USA",2016-02-14 00:00:00+00:00,"{""clock-time"":{""endIndex"":-1,""startIndex"":-1,""...","[{u'injured': True, u'name': u'', u'hospitaliz..."
3,77,"Philadelphia, PA, USA",2015-10-10 00:00:00+00:00,"{""clock-time"":{""endIndex"":-1,""startIndex"":-1,""...","[{u'injured': True, u'name': u'', u'hospitaliz..."
4,80,"Philadelphia, PA, USA",2015-05-02 00:00:00+00:00,"{""clock-time"":{""endIndex"":292,""startIndex"":284...","[{u'injured': True, u'name': u'', u'hospitaliz..."


Each row in the Info about time, type of gun etc. and Info about participants columns are both json like objects. The Info about time, type of gun etc. column is a string with a dictionary and the Info about participants column is a string with a list of dictionaries becuase there can be multiple participants for each event

In [17]:
print(type(phl_events['Info about time, type of gun etc.'][0]))
print(type(phl_events['Info about participants'][0]))
# print the first row in Info about time, type of gun etc column to visually inspect the column more closely
phl_events['Info about time, type of gun etc.'][0]

<class 'str'>
<class 'str'>


'{"clock-time":{"endIndex":177,"startIndex":170,"value":"10 a.m."},"details":{"endIndex":212,"startIndex":185,"value":"1900 block of Morris Street"},"number-of-shots-fired":{"endIndex":345,"startIndex":340,"value":"eight"},"time-day":{"endIndex":-1,"startIndex":-1,"value":""},"type-of-gun":{"endIndex":-1,"startIndex":-1,"value":""}}'

the endIndex and startIndex keys represent the index in the article the event was sourced from. However there is no association to the article. This DB is still in beta. The value key within each dictionary appears to have the most pertinent data  

In [18]:
# extract only the 'value' key/value pair from each row and create a new dataframe
from collections import defaultdict

def extract_keys(dataframe,column):
    value_dict = defaultdict(list)
    dict_column_list = [json.loads(row) for row in dataframe[column]]
    for dict in dict_column_list:
        for k,v in dict.items():
            value_dict[k].append(v['value'])
    cleaned_df = pd.DataFrame.from_dict(value_dict)
    return cleaned_df

In [19]:
info_gun_time = extract_keys(phl_events,'Info about time, type of gun etc.')
print(info_gun_time.info())
info_gun_time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   clock-time             131 non-null    object
 1   details                131 non-null    object
 2   number-of-shots-fired  131 non-null    object
 3   time-day               131 non-null    object
 4   type-of-gun            131 non-null    object
dtypes: object(5)
memory usage: 5.2+ KB
None


,clock-time,details,number-of-shots-fired,time-day,type-of-gun
0,10 a.m.,1900 block of Morris Street,eight,,
1,,,,,
2,,,,,
3,,,,,
4,9:30 p.m,5400 block of Haverford Avenue,,,
...,...,...,...,...,...
126,,60th Street toward Spruce Street,once,,handgun
127,8:30 p.m,100 block of East Clearfield Street,,,
128,,8300 block of Gilbert Street,,midnight,
129,3:25 p.m,700 block of West Luzerne Street,,,


In [20]:
# Unique Values of each column except details 
for col in info_gun_time.columns:
    if col != 'details':
        print(info_gun_time[col].unique())


['10 a.m.' '' '9:30 p.m' '7:30 p.m' '5 p.m.' '12:00 a.m.' '3:00 a.m.'
 '4:30' '4:30 p.m' '12:10 p.m.' '5:47 p.m.' '1:20 p.m.' '1:45 a.m. '
 '4:45 p.m.' '12:35 a.m.' ' 2:45 a.m' '2:55 p.m. ' '3:38 a.m.'
 '4:45 p.m. ' '11:30 p.m' '11 p.m' '6:00 p.m' '10:40 p.m.' '4 a.m'
 '2:30 p.m' '7:15 p.m' '10:45 p.m' '4:05 p.m. ' '12:15 a.m.' '11:30 p.m.'
 '11 p.m.' '5 a.m.' ' 1:30 a.m' '10 a.m' '5:30 a.m' '2:38 p.m' '6:10 p.m.'
 '11:33 p.m' ' 1:30 a.m.' '10:30 a.m. ' '4:10 a.m' '11 p.m. ' '11:50 p.m'
 '11:25 p.m' '3:50 p.m.' ' 12:35 p.m' '12:35 p.m' '9 a.m' '6:30 p.m'
 '1 a.m' '9:30 p.m.' '10:25 p.m.' '3 a.m' ' 6 p.m.' '9 p.m' '1 p.m'
 '3:40 p.m' '1:30 a.m.' '2:15 a.m. ' '3:20 p.m' '2 p.m' '1 p.m. '
 '12:50 p.m' '3:45 a.m.' '11:57 a.m. ' 'noon' '3:30 a.m.' '2:27 p.m'
 '2:30 p.m.' '10:50 ' '1:58 p.m' '2:10 a.m' '6:35 p.m' '12:45 p.m.'
 '10:20 p.m.' '2:20 a.m' '4 p.m' '3:15 a.m' '8:30 p.m.' '1 a.m.' '9:30'
 '10:30 p.m.' '4 a.m.' '7:45 p.m' '9:50 a.m' '8:30 p.m' '3:25 p.m'
 '11:20 p.m.']
['eight' '' ' 

In [21]:
combined = pd.concat([phl_events,info_gun_time],axis =1)
print(combined.shape)
combined.head()

(131, 10)


,index,Address,Date,"Info about time, type of gun etc.",Info about participants,clock-time,details,number-of-shots-fired,time-day,type-of-gun
0,37,"Philadelphia, PA, USA",2015-12-06 00:00:00+00:00,"{""clock-time"":{""endIndex"":177,""startIndex"":170...","[{u'injured': True, u'name': u'', u'hospitaliz...",10 a.m.,1900 block of Morris Street,eight,,
1,48,"Kingsessing, Philadelphia, PA 19143, USA",2016-01-31 00:00:00+00:00,"{""clock-time"":{""endIndex"":-1,""startIndex"":-1,""...","[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,
2,50,"Philadelphia, PA, USA",2016-02-14 00:00:00+00:00,"{""clock-time"":{""endIndex"":-1,""startIndex"":-1,""...","[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,
3,77,"Philadelphia, PA, USA",2015-10-10 00:00:00+00:00,"{""clock-time"":{""endIndex"":-1,""startIndex"":-1,""...","[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,
4,80,"Philadelphia, PA, USA",2015-05-02 00:00:00+00:00,"{""clock-time"":{""endIndex"":292,""startIndex"":284...","[{u'injured': True, u'name': u'', u'hospitaliz...",9:30 p.m,5400 block of Haverford Avenue,,,


In [22]:
combined.drop('Info about time, type of gun etc.', axis = 1, inplace=True)
combined.info()
combined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   index                    131 non-null    int64              
 1   Address                  131 non-null    object             
 2   Date                     131 non-null    datetime64[ns, UTC]
 3   Info about participants  131 non-null    object             
 4   clock-time               131 non-null    object             
 5   details                  131 non-null    object             
 6   number-of-shots-fired    131 non-null    object             
 7   time-day                 131 non-null    object             
 8   type-of-gun              131 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(7)
memory usage: 9.3+ KB


,index,Address,Date,Info about participants,clock-time,details,number-of-shots-fired,time-day,type-of-gun
0,37,"Philadelphia, PA, USA",2015-12-06 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",10 a.m.,1900 block of Morris Street,eight,,
1,48,"Kingsessing, Philadelphia, PA 19143, USA",2016-01-31 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,
2,50,"Philadelphia, PA, USA",2016-02-14 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,
3,77,"Philadelphia, PA, USA",2015-10-10 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,
4,80,"Philadelphia, PA, USA",2015-05-02 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",9:30 p.m,5400 block of Haverford Avenue,,,


In [23]:
import re

def parse_time(df):
    hour = df['clock-time'].str.extract(r'(\d{1,2}):|(\d{1,2})')
    minute = df['clock-time'].str.extract(r':(\d{1,2})')
    pm = df['clock-time'].str.extract(r'(p)')
    
    pm = pm.fillna(0)
    pm.replace({'p':12}, inplace=True)
    pm = pm.astype(int)
    hour = hour.fillna(0)
    minute = minute.fillna(0)
    hour = hour.astype(int)
    minute = minute.astype(int)
    
    hour = hour[0] + hour[1]
    hour = hour + pm[0]
    
    time =pd.concat([hour,minute], keys=['24_hour', 'minute'], axis=1)
    return time
time = parse_time(combined)
time

,24_hour,minute
,0,0
0,10,0
1,0,0
2,0,0
3,0,0
4,21,30
...,...,...
126,0,0
127,20,30
128,0,0


In [24]:
combined = pd.concat([combined, time], axis =1)
combined.head()

,index,Address,Date,Info about participants,clock-time,details,number-of-shots-fired,time-day,type-of-gun,"(24_hour, 0)","(minute, 0)"
0,37,"Philadelphia, PA, USA",2015-12-06 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",10 a.m.,1900 block of Morris Street,eight,,,10,0
1,48,"Kingsessing, Philadelphia, PA 19143, USA",2016-01-31 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,,0,0
2,50,"Philadelphia, PA, USA",2016-02-14 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,,0,0
3,77,"Philadelphia, PA, USA",2015-10-10 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",,,,,,0,0
4,80,"Philadelphia, PA, USA",2015-05-02 00:00:00+00:00,"[{u'injured': True, u'name': u'', u'hospitaliz...",9:30 p.m,5400 block of Haverford Avenue,,,,21,30


In [25]:
phl_events['Info about participants'][0]

"[{u'injured': True, u'name': u'', u'hospitalized': False, u'gender': u'Male', u'age': u'56', u'race': u'', u'killed': False, u'is_victim': True}]"

In [26]:
'''
The items in the Info about participants column are strings with a list of dictionaries inside the string. JSON loads throws an error because the column is single quoted
not double quoted. 
https://docs.python.org/3/library/ast.html#ast.literal_eval
'''
from ast import literal_eval

phl_events['Info about participants'] = phl_events['Info about participants'].apply(literal_eval)



In [27]:
phl_events['Info about participants'][0]

[{'age': '56',
  'gender': 'Male',
  'hospitalized': False,
  'injured': True,
  'is_victim': True,
  'killed': False,
  'name': '',
  'race': ''}]

In [28]:
def get_participants(list_of_dicts):
    info_part_dict = defaultdict(list)
    for row in list_of_dicts:
        for participants in row:
            for key, value in participants.items():
                info_part_dict[key].append(value)
    return info_part_dict   


participant_dict = get_participants(phl_events['Info about participants'])
participant_df = pd.DataFrame.from_dict(participant_dict)

In [29]:
#print(participant_df.info())
print(participant_df)

     injured                name  hospitalized  ... race killed is_victim
0       True                             False  ...       False      True
1       True                              True  ...       False      True
2       True                              True  ...       False      True
3       True                             False  ...       False      True
4       True                              True  ...       False      True
..       ...                 ...           ...  ...  ...    ...       ...
266     True                             False  ...       False      True
267    False      Leotis Martin.         False  ...        True      True
268    False  Bartholomew Nelson         False  ...       False     False
269     True                             False  ...       False      True
270    False                             False  ...       False     False

[271 rows x 8 columns]


In [30]:
articles = pd.read_csv('Articles-with-extracted-info.tsv', sep = '\t')
articles.head()

,Article url,Article title,Full text,Json,Publication date
0,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
1,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00
2,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00
3,gold-data2.com,Man accidentally shoots himself cleaning gun,"BLOOMINGTON, IL - On Thursday evening, Septemb...","{""circumstances"":{""number-of-shots-fired"":{""en...",2013-09-06 00:00:00+00:00
4,gold-data1.com,Man shoots self accidentally while in car,"Atlantic City, Wyo. 26-year-old Joe Henderson ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-01-03 00:00:00+00:00


There is a lot of data that is populated with dummy data. there are approx 7,200 rows with invalid_url

In [31]:
# remove the rows with dummy gold-data url
print(len(articles))
articles = articles[~articles['Article url'].str.contains('gold-data')]
print(len(articles))


articles = articles.reset_index()

23903
16741


In [32]:
articles

,index,Article url,Article title,Full text,Json,Publication date
0,6,http://www.watchfox29.com/story/29662135/polic...,"Police Shoot, Kill Theft Suspect in Houma - FO...",Eight law enforcement officers have been place...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2015-08-03 11:17:57+00:00
1,15,http://www.winknews.com/2015/08/02/12-year-old...,"12-year-old shot three times, suspect on the l...","FORT MYERS, Fla. – A 12-year-old girl is in th...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-08-03 10:24:29+00:00
2,17,http://wreg.com/2015/08/02/mpd-officer-killed-...,MPD Officer killed in the line of duty identif...,"MEMPHIS, Tenn. — Memphis Police have released ...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-08-03 10:24:27+00:00
3,21,http://www.nbc12.com/story/29692846/).text(),Person on bike shot in the Fan - NBC12 - WWBT ...,Richmond Police are looking for suspects after...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2015-08-04 07:27:23+00:00
4,23,http://www.wave3.com/story/29692673/person-sho...,Person shot overnight in Shawnee neighborhood ...,"LOUISVILLE, KY (WAVE) - A person was shot over...","{""circumstances"":{""number-of-shots-fired"":{""en...",2015-08-03 10:54:18+00:00
...,...,...,...,...,...,...
16736,23890,http://www.nbcphiladelphia.com/news/local/Doub...,Philly's Most Dangerous Neighborhood Clocks 2 ...,Neighbors grabbed their kids and made them com...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-08-31 00:00:00+00:00
16737,23891,http://www.nbcphiladelphia.com/news/local/Poli...,New Developments in Murder of Police Officer ...,Philadelphia police said Thursday night there ...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-08-24 00:00:00+00:00
16738,23892,http://www.nbcphiladelphia.com/news/local/Camd...,"Stray Bullets Strike, Kill Woman in Her Home ...",A woman died overnight after apparent stray bu...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-08-16 00:00:00+00:00
16739,23896,http://www.nbcphiladelphia.com/news/local/Pers...,Innocent Bystander Killed After Argument Over ...,Family members say Micah Fisher didn't even li...,"{""circumstances"":{""number-of-shots-fired"":{""en...",2017-11-09 00:00:00+00:00


In [33]:
def clean(articles):
  
  Json_data = articles["Json"].copy()
  dataframe = pd.DataFrame()
  for row in range(16741):
    one_Json = Json_data[row]
    string_form = json.loads(one_Json)
    dict_form = pd.json_normalize(string_form)
    dataframe = dataframe.append(dict_form)
    
  #dataframe.reset_index(inplace = True)  

  return dataframe


In [34]:
dataframe = clean(articles)

In [35]:
dataframe = dataframe.reset_index()

In [36]:
dataframe = dataframe.drop(columns=['index'])
dataframe

,shooter-section,victim-section,circumstances.number-of-shots-fired.endIndex,circumstances.number-of-shots-fired.startIndex,circumstances.number-of-shots-fired.value,circumstances.type-of-gun.endIndex,circumstances.type-of-gun.startIndex,circumstances.type-of-gun.value,date-and-time.city.endIndex,date-and-time.city.startIndex,date-and-time.city.value,date-and-time.clock-time.endIndex,date-and-time.clock-time.startIndex,date-and-time.clock-time.value,date-and-time.date,date-and-time.details.endIndex,date-and-time.details.startIndex,date-and-time.details.value,date-and-time.state,date-and-time.time-day.endIndex,date-and-time.time-day.startIndex,date-and-time.time-day.value,radio1.The firearm was used during another crime.,radio1.The firearm was used in self defense.,radio1.The incident was a case of domestic violence.,radio1.The shooter and the victim knew each other.,radio2.Alcohol was involved.,radio2.Drugs (other than alcohol) were involved.,radio2.The shooting was a suicide or suicide attempt.,radio2.The shooting was self-directed.,radio3.The firearm was owned by the victim/victims family.,radio3.The firearm was stolen.,radio3.The shooting was by a police officer.,radio3.The shooting was directed at a police officer.,radio3.The shooting was unintentional.,date-and-time.details.oldEnd,date-and-time.details.oldStart,circumstances.number-of-shots-fired.oldEnd,circumstances.number-of-shots-fired.oldStart,circumstances.type-of-gun.oldEnd,circumstances.type-of-gun.oldStart,date-and-time.time-day.oldEnd,date-and-time.time-day.oldStart,date-and-time.city.oldEnd,date-and-time.city.oldStart,date-and-time.clock-time.oldEnd,date-and-time.clock-time.oldStart
0,"[{'age': {'endIndex': -1, 'startIndex': -1, 'v...","[{'age': {'endIndex': 99, 'startIndex': 151, '...",-1,-1,,-1,-1,,373,368,Houma,-1,-1,,,576,536,"Officers open fired, killing the suspect",Louisiana,173,159,Monday evening,Not mentioned,Yes,No,No,Not mentioned,Not mentioned,No,No,No,Not mentioned,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'age': {'endIndex': -1, 'startIndex': -1, 'v...","[{'age': {'endIndex': 32, 'startIndex': 21, 'v...",493,477,shot three times,-1,-1,,1089,1079,Fort Myers,-1,-1,,,-1,-1,,Dunbar,226,212,Saturday night,No,No,Yes,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'age': {'endIndex': -1, 'startIndex': -1, 'v...","[{'age': {'endIndex': 151, 'startIndex': 148, ...",-1,-1,,-1,-1,,7,0,MEMPHIS,226,218,9:15 p.m,,209,117,"According to MPD, Sean Bolton, 33, was conduct...",Tenn,-1,-1,,Not mentioned,Not mentioned,No,No,Not mentioned,Not mentioned,No,No,No,Not mentioned,No,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'age': {'endIndex': -1, 'startIndex': -1, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",-1,-1,,-1,-1,,8,0,Richmond,108,92,around 3:15 a.m.,,362,250,The victim says a vehicle ran up on him and o...,,-1,-1,,Not mentioned,No,No,No,Not mentioned,Not mentioned,No,No,No,Not mentioned,No,No,Not mentioned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'age': {'endIndex': -1, 'startIndex': -1, 'v...","[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",-1,-1,,-1,-1,,10,0,LOUISVILLE,-1,-1,,,76,56,Shawnee neighborhood,KY,-1,-1,,Yes,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16736,[],"[{'age': {'endIndex': -1, 'startIndex': -1, 'v...",277,275,18,-1,-1,,245,233,Philadelphia,613,604,1:30 p.m.,2017-08-28,450,319,"Fairhill Square, a park bordered by Huntingdon...",Unclear,596,579,middle of the day,Not mentioned,Not mentioned,No,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentione

In [37]:
def get_philly_data(df):
  philly_df = pd.DataFrame()
  count = 0
  for row in range(len(df.index)):
    if df.iloc[row,10].lower() == 'philadelphia':
      count += 1
      philly_df = philly_df.append(df.iloc[[row]])


  philly_df = philly_df.reset_index()
  philly_df = philly_df.drop(columns=['index'])

  return philly_df

  

None


In [38]:
def extract_shooter(shooter_list):
    names, gender, ages, race = [],[],[],[]
    if len(shooter_list) == 0:
        return pd.Series([names, gender, ages, race])
    for shooter in shooter_list:
        names.append(shooter['name']['value'])
        gender.append(shooter['gender'])
        race.append(shooter['race']['value'])
        shooter_age =shooter['age']['value']
        if  shooter_age== '' or shooter_age is None:
            age=-1
        else:
            age_number = re.search(r'\d+',shooter_age)
            # check if non-empty age values returning valid integer
            if age_number:
                age=int(age_number.group())
            else:
                age=-1
        ages.append(age)
    return pd.Series([names, gender, ages, race])

def extract_victim(victim_list):
    names, gender, ages, race, victim_was = [],[],[],[],[]
    if len(victim_list) == 0:
        return pd.Series([names, gender, ages, race,victim_was])
    for victim in victim_list:
        names.append(victim['name']['value'])
        gender.append(victim['gender'])
        race.append(victim['race']['value'])
        victim_age =victim['age']['value']
        if  victim_age == '' or victim_age is None:
            age=-1
        else:
            age_number = re.search(r'\d+',victim_age)
            # check if non-empty age values returning valid integer
            if age_number:
                age=int(age_number.group())
            else:
                age=-1
        ages.append(age)
        victim_was.append(victim['victim-was'])
    return pd.Series([names, gender, ages, race,victim_was])

In [39]:
dataframe[['victim_names','victim_genders','victim_ages','victim_races','victim_was']] = dataframe['victim-section'].apply(extract_victim)
dataframe[['shooter_names','shooter_genders','shooter_ages','shooter_races']] = dataframe['shooter-section'].apply(extract_shooter)

In [40]:
philly_data = get_philly_data(dataframe)

In [41]:
philly_data = philly_data[['shooter_names','shooter_ages', 'shooter_genders', 'shooter_races', 'victim_names','victim_ages', 'victim_genders', 'victim_races', 'victim_was', 'circumstances.number-of-shots-fired.value', 'circumstances.type-of-gun.value', 'date-and-time.city.value', 'date-and-time.clock-time.value', 'date-and-time.date', 'date-and-time.details.value', 'date-and-time.time-day.value', 'radio1.The firearm was used during another crime.', 'radio1.The firearm was used in self defense.', 'radio1.The incident was a case of domestic violence.', 'radio1.The shooter and the victim knew each other.', 'radio2.Alcohol was involved.', 'radio2.Drugs (other than alcohol) were involved.', 'radio2.The shooting was a suicide or suicide attempt.', 'radio2.The shooting was self-directed.', 'radio3.The firearm was owned by the victim/victims family.', 'radio3.The firearm was stolen.', 'radio3.The shooting was by a police officer.', 'radio3.The shooting was directed at a police officer.', 'radio3.The shooting was unintentional.']]

In [42]:
philly_data

,shooter_names,shooter_ages,shooter_genders,shooter_races,victim_names,victim_ages,victim_genders,victim_races,victim_was,circumstances.number-of-shots-fired.value,circumstances.type-of-gun.value,date-and-time.city.value,date-and-time.clock-time.value,date-and-time.date,date-and-time.details.value,date-and-time.time-day.value,radio1.The firearm was used during another crime.,radio1.The firearm was used in self defense.,radio1.The incident was a case of domestic violence.,radio1.The shooter and the victim knew each other.,radio2.Alcohol was involved.,radio2.Drugs (other than alcohol) were involved.,radio2.The shooting was a suicide or suicide attempt.,radio2.The shooting was self-directed.,radio3.The firearm was owned by the victim/victims family.,radio3.The firearm was stolen.,radio3.The shooting was by a police officer.,radio3.The shooting was directed at a police officer.,radio3.The shooting was unintentional.
0,[],[17],[],[],[Patrick R. Casey],[31],[Male],[],[[injured]],,,Philadelphia,,2015-11-23,I-676,morning,Not mentioned,No,No,Not mentioned,Not mentioned,Not mentioned,No,No,Not mentioned,Not mentioned,No,Yes,No
1,[],[],[],[],[],[11],[Female],[],"[[injured, hospitalized]]",,,Philadelphia,4:30,2015-03-14,Wissinoming section,afternoon,No,No,No,No,No,No,No,No,Yes,No,No,No,Yes
2,[],[],[],[],[],[-1],[Male],[],"[[injured, hospitalized]]",,,Philadelphia,10 p.m,2015-10-23,2300 block of East Ann Street,,Not mentioned,No,No,Not mentioned,Not mentioned,Yes,No,No,No,Not mentioned,No,No,Not mentioned
3,[],[],[],[],[],[-1],[],[],"[[injured, hospitalized]]",,,Philadelphia,,2015-12-20,Kensington,,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,No,Not mentioned
4,[],[],[],[],[],[],[],[],[],,,Philadelphia,,,Germantown,,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,[],[],[],[],[Xavier Stern],[15],[Male],[],[[killed]],,,Philadelphia,,2015-12-13,area of 24th and Park Streets,,Not mentioned,Not mentioned,No,Yes,Not mentioned,Not mentioned,No,No,No,Not mentioned,No,No,No
353,"[Rasheed Scrugs, police]","[-1, -1]","[Male, ]","[, ]","[John Pawlowski, Rasheed Scrugs]","[-1, -1]","[, Male]","[, ]","[[killed], [injured, hospitalized]]",,,Philadelphia,,2009-02-13,,,Not mentioned,Not mentioned,No,No,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Yes,Not mentioned
354,[],[],[],[],"[, ]","[-1, -1]","[, ]","[, ]","[[injured], [injured]]",18,,Philadelphia,1:30 p.m.,2017-08-28,"Fairhill Square, a park bordered by Huntingdon...",middle of the day,Not mentioned,Not mentioned,No,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,No,No,No
355,[],[],[],[],[Moses Walker],[-1],[Male],[],[[killed]],shot three times,,Philadelphia,6 a.m.,2012-08-18,20th and Cecil B. Moore Avenue in North Philad...,,Not mentioned,Not mentioned,No,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Yes,Not mentioned


In [43]:
def colab_to_csv(df, file_name = "data.csv"):

  """Downloads df as CSV"""

  df.to_csv(file_name)
  files.download(file_name)
  return 1

In [44]:
colab_to_csv(combined, "gv_events.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1

In [45]:
colab_to_csv(participant_df, "gv_event_participants.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1